In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import ( basis, expect, mesolve, qeye, sigmax, sigmay, sigmaz,destroy,
                   tensor,fidelity,tracedist,brmesolve,Qobj)
from qutip.solver import heom
from scipy.integrate import quad
from pseudomode import pseudomode,zero_temp_bath,rotation
from hamiltonians import syk_full,plot_ground,plot_fidelities,plot_trd,plot_positivity,plot_populations
from nmm import csolve

In [2]:
N=2
seeds=list(range(42,46))
k=3
H,psis=syk_full(N,seed=seeds[k])

In [3]:
Q=psis[0]

In [4]:
state_list = [basis(2, 1)] + [basis(2, 0)] * (N - 1) # change the initial state to be away from the ground state
psi0 = tensor(state_list)
rho0=psi0*psi0.dag()
H.dims=rho0.dims
Q.dims=rho0.dims
times=np.linspace(0,100,500)
tfit=np.linspace(0, 80, 5000)

In [8]:
oh=heom.OhmicBath(alpha=0.1,wc=1,s=1,T=0,Q=Q) # functions don't take zero because 1/T should be fixed
bath,finfo=oh.make_correlation_fit(tfit,Ni=3,Nr=3)
print(finfo['summary']) #notice one mode is also a pretty good approximation

Fit correlation class instance: 
 
Result of fitting The Real Part Of                                      |Result of fitting The Imaginary Part                                    
 the Correlation Function with 3 terms:                                 | Of the Correlation Function with 3 terms:                              
                                                                        |                                                                        
 Parameters|    a     |    b     |    c                                 | Parameters|    a     |    b     |    c                                 
 1         | 9.80e-02 |-7.94e-01 |4.52e-01                              | 1         |-1.35e-02 |-1.26e+00 |2.37e+00                              
 2         | 5.71e-01 |-1.85e+00 |7.34e-01                              | 2         |-2.91e-01 |-9.24e-01 |6.59e-01                              
 3         |-6.37e-01 |-1.79e+00 |9.05e-04                              | 3         | 2.0

In [ ]:
times2 = np.linspace(0,40,500)
cvis = bath.correlation_function(times2)

In [ ]:
# using the variable axs for multiple Axes
fig, axs = plt.subplots(1, 2,figsize=(15,5))
axs[0].plot(times2, np.imag(cvis),label="Numerical")
axs[0].plot(times2, np.imag(bath1.correlation_function_approx(times2)), "-.",label="Approximated")
axs[0].set_xlabel("t")
axs[0].set_ylabel("Im(C)")
axs[0].legend()
axs[1].plot(times2, np.real(cvis),label="Numerical")
axs[1].plot(times2, np.real(bath1.correlation_function_approx(times2)), "-.",label="Approximated")
axs[1].set_xlabel("t")
axs[1].set_ylabel("Re(C)")
axs[1].legend()
fig.suptitle('Correlation Functions', fontsize=30)
plt.show()

In [9]:
solver = heom.HEOMSolver(H,
                          [bath1], max_depth=7, options={"atol": 1e-14})
result = solver.run(rho0, times)

10.0%. Run time:   2.16s. Est. time left: 00:00:00:19
20.0%. Run time:   3.52s. Est. time left: 00:00:00:14
30.1%. Run time:   4.40s. Est. time left: 00:00:00:10
40.1%. Run time:   5.09s. Est. time left: 00:00:00:07
50.1%. Run time:   5.81s. Est. time left: 00:00:00:05
60.1%. Run time:   6.52s. Est. time left: 00:00:00:04
70.1%. Run time:   7.25s. Est. time left: 00:00:00:03
80.2%. Run time:   7.91s. Est. time left: 00:00:00:01
90.2%. Run time:   8.50s. Est. time left: 00:00:00:00
100.0%. Run time:   9.11s. Est. time left: 00:00:00:00
Total run time:   9.11s


In [10]:
cum = csolve(
    Hsys=H, t=times, baths=[bath],
    Qs=[Q],
    eps=1e-4, cython=False)

result_cum = cum.evolution(rho0)

result_cum = rotation(result_cum, H, times)

/home/mcditoos/github/qutip_gsoc_app/qutip/core/qobj.py:483: RuntimeWarning: divide by zero encountered in scalar divide
  return self.__mul__(1 / other)
Computing Exponential of Generators . . . .: 100%|████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 342.37it/s]


In [11]:
a_ops = [[Q, bath.power_spectrum]]
resultBR = brmesolve(H, rho0, times, a_ops=a_ops, options={
    "atol": 1e-14}, sec_cutoff=-1)

a_ops = [[Q, bath.power_spectrum]]
resultBR2 = brmesolve(H, rho0, times, a_ops=a_ops, options={
    "atol": 1e-14})

In [21]:
Ncutoff=3
modes=2
example = pseudomode(Hsys=H, Q=Q, bath=bath)

In [ ]:
ans = example.evolution(rho0, Ncutoff, times, options={
                        "atol": 1e-14, "normalize_output": False, "store_states": True})
ans = [i.ptrace(range(N))for i in ans.states]

In [ ]:
results=[result,result_cum,resultBR,resultBR2,ans]

In [ ]:
plot_ground(results,H,times)

In [ ]:
plot_fidelities(results,H,times)

In [ ]:
plot_trd(results,H,times)

In [ ]:
plot_positivity(results,H,times)

In [ ]:
plot_populations(results,H,times,l=2,m=2)

In [ ]:
from qutip import qsave,qload

In [ ]:
qsave(results,f"N={N}_syk_{lam}_nocheating_seed_{seeds[k]}")